<a href="https://colab.research.google.com/github/LuHellerKP/SMA_PUCRS/blob/main/Parte_2_%7C_Entrega_%7C_Filas_em_tandem_%5BFEEDBACK%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import random
import heapq

In [9]:
# Função para gerar tempos uniformemente distribuídos
def gerar_tempo(a, b):
    return a + (b - a) * random.random()


In [10]:
# Classe para definir um evento (chegada ou saída)
class Evento:
    def __init__(self, tipo, tempo, fila):
        self.tipo = tipo  # 'CHEGADA' ou 'SAIDA'
        self.tempo = tempo
        self.fila = fila

    def __lt__(self, other):
        return self.tempo < other.tempo

In [11]:
# Fila simulada
class Fila:
    def __init__(self, id, servidores, capacidade, tempo_atendimento, recebe_chegadas=True):
        self.id = id
        self.servidores = servidores  # Número de servidores
        self.capacidade = capacidade  # Capacidade da fila
        self.tempo_atendimento = tempo_atendimento  # Intervalo de tempo para atendimento
        self.recebe_chegadas = recebe_chegadas  # Se a fila recebe clientes de fora
        self.clientes_na_fila = 0
        self.servidores_ocupados = 0
        self.clientes_perdidos = 0
        self.tempo_acumulado = {i: 0 for i in range(capacidade + 1)}  # Tempo acumulado para cada estado da fila

In [12]:
# Simulador de rede de filas em tandem
class SimuladorRedeFilas:
    def __init__(self, filas):
        self.filas = filas  # Lista de filas
        self.eventos = []  # Fila de eventos
        self.tempo_atual = 0
        self.tempo_simulacao = 0

    def adicionar_evento(self, tipo, tempo, fila):
        heapq.heappush(self.eventos, Evento(tipo, tempo, fila))

    def processar_evento(self, evento):
        self.tempo_atual = evento.tempo
        fila = evento.fila

        # Atualizar tempo acumulado
        fila.tempo_acumulado[fila.clientes_na_fila] += (self.tempo_atual - self.tempo_simulacao)
        self.tempo_simulacao = self.tempo_atual

        if evento.tipo == 'CHEGADA':
            self.processar_chegada(fila)
        elif evento.tipo == 'SAIDA':
            self.processar_saida(fila)

    def processar_chegada(self, fila):
        if fila.clientes_na_fila < fila.capacidade:
            fila.clientes_na_fila += 1
            if fila.servidores_ocupados < fila.servidores:
                fila.servidores_ocupados += 1
                tempo_atendimento = gerar_tempo(*fila.tempo_atendimento)
                self.adicionar_evento('SAIDA', self.tempo_atual + tempo_atendimento, fila)
            # Se a fila recebe chegadas externas, agenda próxima chegada
            if fila.recebe_chegadas:
                tempo_chegada = gerar_tempo(1.0, 4.0)
                self.adicionar_evento('CHEGADA', self.tempo_atual + tempo_chegada, fila)
        else:
            fila.clientes_perdidos += 1

    def processar_saida(self, fila):
        if fila.clientes_na_fila > 0:
            fila.clientes_na_fila -= 1
            fila.servidores_ocupados -= 1
            if fila.clientes_na_fila >= fila.servidores_ocupados:
                tempo_atendimento = gerar_tempo(*fila.tempo_atendimento)
                self.adicionar_evento('SAIDA', self.tempo_atual + tempo_atendimento, fila)

        # Se há uma fila seguinte (tandem), cliente vai para a próxima fila
        if fila.id == 1:
            self.adicionar_evento('CHEGADA', self.tempo_atual, self.filas[1])

    def executar(self, num_eventos):
        # Inicializa com a primeira chegada na Fila 1
        self.adicionar_evento('CHEGADA', 1.5, self.filas[0])

        while len(self.eventos) > 0 and num_eventos > 0:
            evento = heapq.heappop(self.eventos)
            self.processar_evento(evento)
            num_eventos -= 1

    def relatorio(self):
        for fila in self.filas:
            print(f"Fila {fila.id}:")
            print(f"  Clientes perdidos: {fila.clientes_perdidos}")
            for i in range(fila.capacidade + 1):
                probabilidade = fila.tempo_acumulado[i] / self.tempo_simulacao
                print(f"  {i} clientes: {probabilidade:.4f}")
            print()

In [13]:
# Configuração das filas
fila_1 = Fila(id=1, servidores=2, capacidade=3, tempo_atendimento=(3.0, 4.0), recebe_chegadas=True)
fila_2 = Fila(id=2, servidores=1, capacidade=5, tempo_atendimento=(2.0, 3.0), recebe_chegadas=False)


In [14]:
# Simulação
simulador = SimuladorRedeFilas([fila_1, fila_2])
simulador.executar(100000)
simulador.relatorio()

Fila 1:
  Clientes perdidos: 1
  0 clientes: 0.2713
  1 clientes: 0.1418
  2 clientes: 0.0199
  3 clientes: 0.0008

Fila 2:
  Clientes perdidos: 0
  0 clientes: 0.2567
  1 clientes: 0.1921
  2 clientes: 0.0966
  3 clientes: 0.0190
  4 clientes: 0.0018
  5 clientes: 0.0001



Descrição:
Fila 1 (G/G/2/3):

Servidores: 2
Capacidade: 3 clientes
Chegadas: Intervalo entre 1 e 4 minutos
Atendimento: Intervalo entre 3 e 4 minutos

Fila 2 (G/G/1/5):

Servidores: 1
Capacidade: 5 clientes
Atendimento: Intervalo entre 2 e 3 minutos
Sem chegadas externas, recebe clientes da Fila 1.